In [20]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')





In [21]:
from clustering import *

In [22]:
df_clusters=pd.read_csv("../Data/Processed/nse_features.csv")
df_clusters.head(5)

,Stock_code,Sector,Name,trading_days,total_days,trading_frequency,mean_return,std_return,volatility_7d,volatility_30d,max_drawdown,avg_volume,zero_volume_ratio,momentum_30d,current_price
0,ABSA,Banking,ABSA Bank Kenya Plc,988,988,1.000000,0.000743,0.013638,0.011135,0.012541,-0.236220,457918.016194,0.000000,0.160772,18.05
1,BAMB,Construction and Allied,Bamburi Cement Ltd,974,988,0.985830,0.000901,0.028007,0.021978,0.024157,-0.514563,233655.667351,0.014170,-0.176030,55.00
2,BAT,Manufacturing and Allied,British American Tobacco Kenya Plc,936,988,0.947368,0.000126,0.013762,0.010576,0.012213,-0.348232,19173.611111,0.052632,0.059155,376.00
3,BKG,Banking,BK Group Plc,592,988,0.599190,0.001923,0.038372,0.025778,0.028502,-0.392500,86221.114865,0.400810,-0.012140,32.55
4,BOC,Manufacturing and Allied,BOC Kenya Plc,402,988,0.406883,0.001758,0.042706,0.021544,0.026053,-0.302949,10246.268657,0.593117,-0.002809,88.75


In [23]:
df_clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Stock_code         57 non-null     object 
 1   Sector             57 non-null     object 
 2   Name               57 non-null     object 
 3   trading_days       57 non-null     int64  
 4   total_days         57 non-null     int64  
 5   trading_frequency  57 non-null     float64
 6   mean_return        57 non-null     float64
 7   std_return         57 non-null     float64
 8   volatility_7d      57 non-null     float64
 9   volatility_30d     57 non-null     float64
 10  max_drawdown       57 non-null     float64
 11  avg_volume         57 non-null     float64
 12  zero_volume_ratio  57 non-null     float64
 13  momentum_30d       57 non-null     float64
 14  current_price      57 non-null     float64
dtypes: float64(10), int64(2), object(3)
memory usage: 6.8+ KB


In [24]:
feature_cols = [
    'volatility_30d',
    'return_std',
    'max_drawdown',
    'avg_volume',
    'zero_volume_ratio',
    'trading_frequency',
    'volatility_vs_sector'
]

# Keeping only available features
feature_cols = [col for col in feature_cols if col in df_clusters.columns]
print(f"Features for clustering: {feature_cols}")

Features for clustering: ['volatility_30d', 'max_drawdown', 'avg_volume', 'zero_volume_ratio', 'trading_frequency']


In [25]:
cluster_eval = find_optimal_clusters(df_clusters, feature_cols, max_clusters=6)
print(cluster_eval)

   n_clusters     inertia  silhouette
0           2  192.994719    0.415230
1           3  138.013945    0.436023
2           4   89.442090    0.342952
3           5   67.563011    0.356603
4           6   52.232691    0.329345
